In [13]:
import torch
import torchvision
import torchvision.models as models
from torchvision import transforms
from PIL import Image
import torch.optim as optim
import timm
import torch.nn as nn


In [3]:
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]

In [4]:
train_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.RandomAffine(0, shear=10, scale=(0.8, 1.2)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(30),
        transforms.ToTensor(),
        transforms.Normalize(mean=means, std=stds),
    ]
)
val_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=means, std=stds),
    ]
)

In [5]:
import os
from torch.utils.data import Dataset


class WheatDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.classes = [x.split("_")[1] for x in sorted(os.listdir(data_dir))]
        self.class_to_idx = {cls: i for i, cls in enumerate(self.classes)}

        for label_dir in sorted(os.listdir(data_dir)):
            label_name = label_dir.split("_")[1]
            pth = os.path.join(data_dir, label_dir)
            for img_name in os.listdir(pth):
                self.image_paths.append(os.path.join(pth, img_name))
                self.labels.append(self.class_to_idx[label_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]

        if self.transform:
            image = self.transform(image)

        return image, label

In [6]:
from torch.utils.data import DataLoader

train_dataset = WheatDataset(
    data_dir="/home/sergei/Downloads/GrainSetData/wheat/train",
    transform=train_transforms,
)
val_dataset = WheatDataset(
    data_dir="/home/sergei/Downloads/GrainSetData/wheat/test", transform=val_transforms
)


In [9]:
WORKERS = 16
NUM_CLASSES = 8

In [10]:
train_loader = DataLoader(
    train_dataset,
    batch_size=180,
    shuffle=True,
    num_workers=WORKERS,
    pin_memory=True,
    drop_last=True,
)
val_loader = DataLoader(
    val_dataset,
    batch_size=180,
    shuffle=False,
    num_workers=WORKERS,
    pin_memory=True,
    drop_last=True,
)


In [20]:
model_names = timm.list_models(pretrained=True)

In [21]:
model_names

['aimv2_1b_patch14_224.apple_pt',
 'aimv2_1b_patch14_336.apple_pt',
 'aimv2_1b_patch14_448.apple_pt',
 'aimv2_3b_patch14_224.apple_pt',
 'aimv2_3b_patch14_336.apple_pt',
 'aimv2_3b_patch14_448.apple_pt',
 'aimv2_huge_patch14_224.apple_pt',
 'aimv2_huge_patch14_336.apple_pt',
 'aimv2_huge_patch14_448.apple_pt',
 'aimv2_large_patch14_224.apple_pt',
 'aimv2_large_patch14_224.apple_pt_dist',
 'aimv2_large_patch14_336.apple_pt',
 'aimv2_large_patch14_336.apple_pt_dist',
 'aimv2_large_patch14_448.apple_pt',
 'bat_resnext26ts.ch_in1k',
 'beit3_base_patch16_224.in22k_ft_in1k',
 'beit3_base_patch16_224.indomain_in22k_ft_in1k',
 'beit3_base_patch16_224.indomain_pt',
 'beit3_base_patch16_224.pt',
 'beit3_large_patch16_224.in22k_ft_in1k',
 'beit3_large_patch16_224.indomain_in22k_ft_in1k',
 'beit3_large_patch16_224.indomain_pt',
 'beit3_large_patch16_224.pt',
 'beit_base_patch16_224.in22k_ft_in22k',
 'beit_base_patch16_224.in22k_ft_in22k_in1k',
 'beit_base_patch16_384.in22k_ft_in22k_in1k',
 'beit_l

In [37]:
model = timm.create_model("resnet10t", pretrained=True, num_classes=NUM_CLASSES)

model.safetensors:   0%|          | 0.00/21.8M [00:00<?, ?B/s]

In [39]:
# 1. Freeze all parameters
for param in model.parameters():
    param.requires_grad = False

# 2. Unfreeze final layer
for param in model.fc.parameters():
    param.requires_grad = True

In [40]:
learn_rate = 1.2e-4
wd = 1e-4

In [41]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=learn_rate, weight_decay=wd)


In [42]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [43]:
model.to(device)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (drop_block): Identity()
   

In [ ]:
# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    print(f"Epoch = {epoch}")
    # Training phase
    model.train()  # Set the model to training mode
    running_loss = 0.0
    correct = 0
    total = 0

    # Iterate over the training data
    for inputs, labels in train_loader:
        # Move data to the specified device (GPU or CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients to prevent accumulation
        optimizer.zero_grad()
        # Forward pass
        outputs = model(inputs)
        # Calculate the loss
        loss = criterion(outputs, labels)
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate training loss
        running_loss += loss.item()
        # Get predictions
        _, predicted = torch.max(outputs.data, 1)
        # Update total and correct predictions
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        print("One iteration")

    # Calculate average training loss and accuracy
    train_loss = running_loss / len(train_loader)
    train_acc = correct / total

    # Validation phase
    model.eval()  # Set the model to evaluation mode
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    # Disable gradient calculation for validation
    with torch.no_grad():
        # Iterate over the validation data
        for inputs, labels in val_loader:
            # Move data to the specified device (GPU or CPU)
            inputs, labels = inputs.to(device), labels.to(device)
            # Forward pass
            outputs = model(inputs)
            # Calculate the loss
            loss = criterion(outputs, labels)

            # Accumulate validation loss
            val_loss += loss.item()
            # Get predictions
            _, predicted = torch.max(outputs.data, 1)
            # Update total and correct predictions
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    # Calculate average validation loss and accuracy
    val_loss /= len(val_loader)
    val_acc = val_correct / val_total

    # Print epoch results
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print(f"  Train Loss: {train_loss:.4f}, Train Acc: {train_acc:.4f}")
    print(f"  Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.4f}")

Epoch = 0
One iteration
One iteration
One iteration
One iteration
One iteration
